In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
pharm_data = pd.read_csv("./data/Pharm Data_Data.csv")
pharm_description = pd.read_csv("./data/Pharm Data_Fields Descriptions.csv")
pharm_data.head()

,Distributor,Customer Name,City,Country,Latitude,Longitude,Channel,Sub-channel,Product Name,Product Class,Quantity,Price,Sales,Month,Year,Name of Sales Rep,Manager,Sales Team
0,Gottlieb-Cruickshank,"Zieme, Doyle and Kunze",Lublin,Poland,51.2333,22.5667,Hospital,Private,Topipizole,Mood Stabilizers,4.0,368,1472.0,January,2018,Mary Gerrard,Britanny Bold,Delta
1,Gottlieb-Cruickshank,Feest PLC,Świecie,Poland,53.4167,18.4333,Pharmacy,Retail,Choriotrisin,Antibiotics,7.0,591,4137.0,January,2018,Jessica Smith,Britanny Bold,Delta
2,Gottlieb-Cruickshank,Medhurst-Beer Pharmaceutical Limited,Rybnik,Poland,50.0833,18.5000,Pharmacy,Institution,Acantaine,Antibiotics,30.0,66,1980.0,January,2018,Steve Pepple,Tracy Banks,Bravo
3,Gottlieb-Cruickshank,Barton Ltd Pharma Plc,Czeladź,Poland,50.3333,19.0833,Hospital,Private,Lioletine Refliruvax,Analgesics,6.0,435,2610.0,January,2018,Mary Gerrard,Britanny Bold,Delta
4,Gottlieb-Cruickshank,Keeling LLC Pharmacy,Olsztyn,Poland,53.7800,20.4942,Pharmacy,Retail,Oxymotroban Fexoformin,Analgesics,20.0,458,9160.0,January,2018,Anne Wu,Britanny Bold,Delta


In [3]:
pharm_reviews_train = pd.read_csv("./data/drugsComTrain_raw.csv")
pharm_reviews_test = pd.read_csv("./data/drugsComTest_raw.csv")
pharm_reviews_train.head()

,uniqueID,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,3-Nov-15,10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37


In [8]:
drugname1 = pharm_data['Product Name'].unique()
drugname2 = pharm_reviews_train['drugName'].unique()
for drug in drugname1:
    if drug not in drugname2:
        print("no:",drug)
    else:
        print("yes:",drug)


no: Topipizole
no: Choriotrisin
no: Acantaine
no: Lioletine Refliruvax
no: Oxymotroban Fexoformin
no: Pazofenac
no: Symbitrim
no: Morphizolid Tianalin
no: Lovapur
no: Ampysin
no: Trazobalamin
no: Atrabicin Alkerotec
no: Exotropin Empizine
no: Afinitasol
no: Amavirase
no: Neuropogen Empibax
no: Tracdomide
no: Novakyn Tracprox
no: Paronium Atracustone
no: Dantocept Ferurenone
no: Aciprex
no: Diaxolol
no: Saprodeine Zanprosyn
no: Magnemunex
no: Pitodomide
no: Malaxolol Aeroprosyn
no: Tetratanyl
no: Spiroderall
no: Formolovir Amanferon
no: Panpion
no: Dorzovorin Pentadarone
no: Comzyme
no: Aldevac
no: Invisteride Diclomycin
no: Ergomethate
no: Theopril
no: Ketastadil
no: Methiclotide
no: Propratecan
no: Lovavatol Azelavarix
no: Symbiroban
no: Afaxacin
no: Testant
no: Temasone Thiobucil
no: Propralamide Dexlanzolam
no: Abobozolid
no: Choriogestrel
no: Factofribrate
no: Sanctumine
no: Epzipitant
no: Relenstrel Exurabine
no: Kiovax Tercorolac
no: Biotasol Siloderm
no: Cogxone Levanide
no: Fin

In [12]:
with open("./data/drugname1.txt", "w") as f:
    for drug in drugname1:
        f.write(drug + "\n")
with open("./data/drugname2.txt", "w") as f:
    for drug in drugname2:
        f.write(drug + "\n")

In [13]:
pharm_data['Product Class'].unique()

array(['Mood Stabilizers', 'Antibiotics', 'Analgesics', 'Antiseptics',
       'Antipiretics', 'Antimalarial'], dtype=object)

In [16]:
import json

with open("./data/drugclasses.jsonl", "w") as f:
    for i in range(len(drugname2)):
        drugname = drugname2[i]
        template = {"custom_id": f"request-{i}", "body": {"messages": [{"role": "user", "content": f"classify the drug name {drugname} to one of ['Mood Stabilizers', 'Antibiotics', 'Analgesics', 'Antiseptics', 'Antipiretics', 'Antimalarial', 'Else'], you only need to return the classification result such as '(drugname,class)' "}],"max_tokens": 20,"top_p":1}}
        f.write(json.dumps(template) + "\n")


In [20]:
# using openai api to classify the drug names
'''import os
from openai import OpenAI
from tqdm import tqdm

client = OpenAI(
    # 此为默认路径，您可根据业务所在地域进行配置
    base_url="https://ark.cn-beijing.volces.com/api/v3",
    # 您的 API Key
    api_key="xxxxxxxxxxxxxxxxxxxxxxx",
)

# Non-streaming:
print("----- standard request -----")
drugname2_classes = []
for i in tqdm(range(len(drugname2))):
    drugname = drugname2[i]
    completion = client.chat.completions.create(
        # 指定您创建的方舟推理接入点 ID，此处已帮您修改为您的推理接入点 ID
        model="ep-20250405115326-tzr72",
        messages=[
            {"role": "system", "content": "you are a medical drug classification assistant"},
            {"role": "user", "content": f"classify the drug name {drugname} to one of ['Mood Stabilizers', 'Antibiotics', 'Analgesics', 'Antiseptics', 'Antipiretics', 'Antimalarial', 'Else'], you only need to return the classification result such as '(drugname,class)' "}
        ],
        max_tokens=20,
        top_p=1,
    )
    # print(completion.choices[0].message.content)
    drugname2_classes.append(completion.choices[0].message.content)'''

----- standard request -----


100%|██████████| 3436/3436 [59:56<00:00,  1.05s/it]  


In [52]:
# with open("./data/drugname2_classes.txt", "w") as f:
#     for info in drugname2_classes:
#         f.write(info + "\n")
# 创建一个空字典来存储pair
pair_dict = {}

# 打开文件并读取内容
with open('./data/drugname2_classes.txt', 'r') as file:
    cnt = 0
    for line in file:
        cnt += 1
        # 去除行尾的换行符，并按逗号分割
        try:
            key, value = line.strip().split(',')
            # 将分割后的键值对添加到字典中
            pair_dict[key.strip('(').strip(')')] = value.strip(')')
        except:
            print(cnt)

# 打印字典查看结果
# print(pair_dict)


In [53]:
# add column 'Product Class' to pharm_reviews_train due to pair_dict, fill na with 'Else'
pharm_reviews_train['Product Class'] = pharm_reviews_train['drugName'].map(pair_dict)
pharm_reviews_train['Product Class'] = pharm_reviews_train['Product Class'].fillna('Else')

In [54]:
pharm_reviews_test['Product Class'] = pharm_reviews_test['drugName'].map(pair_dict)
pharm_reviews_test['Product Class'] = pharm_reviews_test['Product Class'].fillna('Else')

In [56]:
# save the data to csv
pharm_reviews_train.to_csv("./data/drugsComTrain_raw_addclass.csv", index=False)
pharm_reviews_test.to_csv("./data/drugsComTest_raw_addclass.csv", index=False)

In [58]:
pharm_reviews_train['Product Class'].value_counts()

Product Class
Else                115500
Analgesics           16390
Mood Stabilizers     15342
Antibiotics          10124
Antiseptics           3580
Antimalarial           249
Antipiretics           112
Name: count, dtype: int64

In [59]:
pharm_data['Product Class'].value_counts()

Product Class
Antiseptics         52037
Mood Stabilizers    46415
Analgesics          44751
Antibiotics         36979
Antipiretics        36955
Antimalarial        36945
Name: count, dtype: int64